Video UI Demo: 

In [14]:
# importing libraries
import os
import collections
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import cv2

# Setup

In [2]:
vision_encoder = keras.models.load_model("vision_encoder")
text_encoder = keras.models.load_model("text_encoder")

In [3]:
vision_encoder.compile()
text_encoder.compile()

I have the mp4 files and transcription text files saved in my working directory from Milestone 1. I will extract a set amount of frames from each video adhering to the dimensions required by the vision encoder; These frames will then be converted into embeddings. Then, I will convert the text into embeddings. Each embedding will be saved into an .npy file.

In [15]:
os.makedirs("embeddings")

In [12]:
mp4_files = [file for file in os.listdir(os.getcwd()) if file.endswith(".mp4")]
text_files = [file for file in os.listdir(os.getcwd()) if file.endswith(".txt")]

In [39]:
for file in mp4_files:
    cap = cv2.VideoCapture(file)
    frameCount = 0
    extractedFrames = 10
    totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    skipFactor = int(totalFrames / extractedFrames)
    while cap.isOpened:
        ret, frame = cap.read()          
        if not ret:
            break
        frameCount += 1
        if frameCount % skipFactor != 0:
            continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (299, 299))
        frame = np.expand_dims(frame, axis=0)
        embedding = vision_encoder.predict(frame)
        fileName = file.split(".")[0] + "-" + str(int(frameCount/skipFactor)) + ".npy"
        np.save("embeddings/" + fileName, embedding)

1/1 [==============================] - 0s 155ms/step


1/1 [==============================] - 0s 148ms/step


In [70]:
for file in text_files:
    inFile = open(file, "r")
    lines = inFile.readlines()
    inFile.close()
    txt = ""
    for line in lines:
        if "->" in line or line == "\n":
            continue
        else:
            txt += line
    tokens = txt.split()
    embedding = text_encoder.predict(tokens)
    fileName = file.split(".")[0] + "-txt" + ".npy"
    np.save("embeddings/" + fileName, embedding)

17/17 [==============================] - 10s 576ms/step


After the npy files were saved to my drive, I created a PHP file that handles the web requests. The user sends a POST request through the form with their query as a parameter in the request body. PHP handles this request by executing a Python script that converts the query into an embedding. Then, all the embeddings we have stored are iterated through, allowing us to calculate the cosine similarity between the generated embedding and the stored embeddings. The 3 unique video id's with the highest similarity score are then displayed to the user. See the index.php and backend.py files on GitHub to see the implementation of these.